# Part 7 - Federated Learning with FederatedDataset

Here we introduce a new tool for batching federated datasets.

In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Setting up Sandbox...
Done!


In [2]:
boston_data, _ = grid.search("#boston", "#data", verbose=False)
boston_target, _ = grid.search("#boston", "#target", verbose=False)

In [3]:
n_inputs = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_inputs,n_targets, bias=False)
opt = sy.optim.SGD(params=model.parameters(),lr=0.0000001)

fd = sy.FederatedDataset(boston_data, boston_target, batch_size=32, num_iterators=1)

In [7]:
for iter in range(100):
    fd.reset()
    i = 0
    loss_accum = 0 
    while(fd.keep_going()):
        i += 1

        data, target = fd.step()
        model.send(data.location)
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step(data.shape[0])
        model.get()
        loss_accum += float(loss.get())
    if(iter % 10 == 0):
        print("iter:" + str(iter) + " loss:" + str(loss_accum / i))

iter:0 loss:137343.05072699653
iter:10 loss:136191.96234809028
iter:20 loss:135099.33181423612
iter:30 loss:134061.3917643229
iter:40 loss:133074.57590060763
iter:50 loss:132135.70024956597
iter:60 loss:131241.6998155382
iter:70 loss:130389.80230034722
iter:80 loss:129577.5292154948
iter:90 loss:128802.40188259548
